In [1]:
import arrow
import librosa
import os
import pandas as pd
from glob import glob

PATHNAME = '/kaggle/input/birdclef-2024/train_audio/*/*.ogg'
RATE = 22050

time_start = arrow.now()
data = []
for input_file in glob(pathname=PATHNAME, recursive=True):
    folder = input_file.split('/')[-2]
    name = os.path.basename(input_file)
    audio, _ = librosa.load(input_file, sr=RATE)
    features = librosa.feature.mfcc(y=audio, sr=RATE, n_mfcc=1)
    data.append(pd.Series(data=['folder', name, features[0]], index=['species', 'file', 'data']))
    
df = pd.DataFrame(data=data)
df['length'] = df['data'].apply(func=len)

print('{} done'.format(arrow.now() - time_start))

1:24:00.738673 done


In [2]:
from plotly import express

express.histogram(data_frame=df, x='length', color='species')